# １章 ウェブの注文数を分析する１０本ノック

ここでは、ある企業のECサイトでの商品の注文数の推移を分析していきます。  
データの属性を理解し、分析をするためにデータを加工した後、  
データの可視化を行うことで問題を発見していくプロセスを学びます。

### ノック１：データを読み込んでみよう

In [1]:
!pip list

Package                Version            
---------------------- -------------------
alembic                1.4.1              
async-generator        1.10               
attrs                  19.3.0             
backcall               0.1.0              
beautifulsoup4         4.8.2              
bleach                 3.1.3              
blinker                1.4                
bokeh                  1.4.0              
certifi                2019.11.28         
certipy                0.1.3              
cffi                   1.14.0             
chardet                3.0.4              
click                  7.1.1              
cloudpickle            1.3.0              
conda                  4.8.2              
conda-package-handling 1.6.0              
cryptography           2.8                
cycler                 0.10.0             
Cython                 0.29.15            
cytoolz                0.10.1             
dask                   2.11.0             
decorator  

In [8]:
import os
os.getcwd()

'/code/practice_code/chapter_1'

In [2]:
import pandas as pd

In [10]:
sample_code_dir = '/code/sample_code/1章/'

In [3]:
customer_master = pd.read_csv('../../sample_code/1章/customer_master.csv')
display(customer_master.head(), customer_master.shape)

,customer_id,customer_name,registration_date,customer_name_kana,email,gender,age,birth,pref
0,IK152942,平田 裕次郎,2019-01-01 00:25:33,ひらた ゆうじろう,hirata_yuujirou@example.com,M,29,1990/6/10,石川県
1,TS808488,田村 詩織,2019-01-01 01:13:45,たむら しおり,tamura_shiori@example.com,F,33,1986/5/20,東京都
2,AS834628,久野 由樹,2019-01-01 02:00:14,ひさの ゆき,hisano_yuki@example.com,F,63,1956/1/2,茨城県
3,AS345469,鶴岡 薫,2019-01-01 04:48:22,つるおか かおる,tsuruoka_kaoru@example.com,M,74,1945/3/25,東京都
4,GD892565,大内 高史,2019-01-01 04:54:51,おおうち たかし,oouchi_takashi@example.com,M,54,1965/8/5,千葉県


(5000, 9)

In [32]:
display(customer_master['customer_id'].unique)

<bound method Series.unique of 0       IK152942
1       TS808488
2       AS834628
3       AS345469
4       GD892565
          ...   
4995    AS677229
4996    HD758694
4997    PL538517
4998    OA955088
4999    HI349563
Name: customer_id, Length: 5000, dtype: object>

In [13]:
item_master = pd.read_csv('/code/sample_code/1章/item_master.csv')
display(item_master.head(), item_master.shape)

,item_id,item_name,item_price
0,S001,PC-A,50000
1,S002,PC-B,85000
2,S003,PC-C,120000
3,S004,PC-D,180000
4,S005,PC-E,210000


(5, 3)

In [12]:
transaction_1 = pd.read_csv(sample_code_dir + 'transaction_1.csv')
display(transaction_1.head(), transaction_1.shape)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865


(5000, 4)

In [15]:
transaction_detail_1 = pd.read_csv(sample_code_dir + 'transaction_detail_1.csv')
display(transaction_detail_1.head(), transaction_detail_1.shape)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2


(5000, 4)

### ノック２：データを結合(ユニオン)してみよう

In [17]:
transaction_2 = pd.read_csv(sample_code_dir + 'transaction_2.csv')
display(transaction_2.shape)
# ignore_indexでindex番号を振り直す。 axisで縦に結合を指示。
transaction = pd.concat([transaction_1, transaction_2], ignore_index=True, axis=0)
display(transaction.head(), transaction.shape)
transaction_zikken = pd.concat([transaction_1, transaction_2], axis=1)
display(transaction_zikken.head(), transaction_zikken.shape)

(1786, 4)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865


(6786, 4)

,transaction_id,price,payment_date,customer_id,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,T0000005113,295000.0,2019-06-15 07:20:27,TS169261
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,T0000005114,50000.0,2019-06-15 07:35:47,HI599892
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,T0000005115,85000.0,2019-06-15 07:56:36,HI421757
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,T0000005116,50000.0,2019-06-15 08:40:55,OA386378
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,T0000005117,120000.0,2019-06-15 08:44:23,TS506913


(5000, 8)

In [18]:
transaction_detail_2 = pd.read_csv(sample_code_dir + 'transaction_detail_2.csv')
display(transaction_detail_2.shape)
# ignore_indexでindex番号を振り直す。 axisで縦に結合を指示。
transaction_detail = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index=True, axis=0)
display(transaction_detail.head(), transaction_detail.shape)

(2144, 4)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2


(7144, 4)

### ノック３：売上データ同士を結合(ジョイン)してみよう

In [21]:
## １つのDFに1つのDFをmergeする時にはmerge。 マージしたいkeyも指定可能。 onにてマージする主軸を決める。 howでマージの仕方(今回右に入れた)
join_data = pd.merge(transaction_detail, transaction[["transaction_id", "payment_date", "customer_id"]], on="transaction_id", how="left")
display(join_data.head(), join_data.shape)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865


(7144, 6)

### ノック４：マスタデータを結合(ジョイン)してみよう

In [33]:
join_data = pd.merge(join_data, customer_master, on="customer_id", how="left")
join_data = pd.merge(join_data, item_master, on="item_id", how="left")
display(join_data.head(), join_data.shape)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,customer_name_kana,email,gender,age,birth,pref,item_name,item_price
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,井本 芳正,2019-01-07 14:34:35,いもと よしまさ,imoto_yoshimasa@example.com,M,30,1989/7/15,熊本県,PC-E,210000
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019,三船 六郎,2019-01-27 18:00:11,みふね ろくろう,mifune_rokurou@example.com,M,73,1945/11/29,京都府,PC-A,50000
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120,山根 小雁,2019-01-11 08:16:02,やまね こがん,yamane_kogan@example.com,M,42,1977/5/17,茨城県,PC-C,120000
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215,池田 菜摘,2019-01-10 05:07:38,いけだ なつみ,ikeda_natsumi@example.com,F,47,1972/3/17,兵庫県,PC-E,210000
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865,栗田 憲一,2019-01-25 06:46:05,くりた けんいち,kurita_kenichi@example.com,M,74,1944/12/17,長崎県,PC-B,85000


(7144, 16)

### ノック5：必要なデータ列を作ろう

### ノック6：データ検算をしよう

### ノック7：各種統計量を把握しよう

### ノック8：月別でデータを集計してみよう

### ノック9：月別、商品別でデータを集計してみよう

### ノック10：商品別の売上推移を可視化してみよう